# 介绍 N2@Cu

计算N2在Cu上的吸附能:   
$$E_{ads} = E_{N2} + E_{Cu} - E_{N2@Cu}$$  
* $E_{N2}, E_{Cu}$: N2和Cu单独存在时的能量;   
* $E_{N2@Cu}$: N2吸附在Cu上时的总能;  

In [1]:
from ase import Atoms
from ase.visualize import view
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate

## Atoms

**Atoms**: 原子集(合)对象

In [2]:
# 创建分子对象
from ase import Atoms
d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule

Atoms(symbols='N2', pbc=False)

使用点阵模块(lattice module)构建晶体, 晶体也是一个原子集对象

In [3]:
# 创建slab对象
from ase.build import fcc111
slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)
slab 

Atoms(symbols='Cu32', pbc=[True, True, False], cell=[[10.210621920333747, 0.0, 0.0], [5.105310960166873, 8.842657971447272, 0.0], [0.0, 0.0, 22.08423447177455]], tags=...)

## Calculators

ASE可使用的能量计算器: emt(ase内嵌,用于演示功能), Vasp(正规)..

In [4]:
from ase.calculators.emt import EMT
slab.set_calculator(EMT())
molecule.set_calculator(EMT())

原子集对象经过能量计算器之后, 可以调用能量查询方法, get_potential_energy()

In [5]:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()

## Structure relaxation

在Cu的top位上吸附N2

In [6]:
h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

为了加快速度, 只允许N2分子松弛, 而将基地的Cu原子固定

In [7]:
from ase.constraints import FixAtoms
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

使用准牛顿算法对吸附构型进行能量最小化, 并保存轨迹文件

In [8]:
from ase.optimize import QuasiNewton
dyn = QuasiNewton(slab, trajectory='data/N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 21:54:58       11.689927*       1.0797
BFGSLineSearch:    1[  2] 21:54:58       11.670814*       0.4090
BFGSLineSearch:    2[  4] 21:54:58       11.625880*       0.0409


True

In [9]:
e_slab_N2 = slab.get_potential_energy()

In [10]:
%C e_slab; e_N2; e_slab_N2; e_slab+e_N2-e_slab_N2

      e_slab               e_N2             e_slab_N2       e_slab+e_N2-e_slab_N2
-----------------  -------------------  ------------------  ---------------------
11.50905628357027  0.44034357303561467  11.625880434287794  0.32351942231809083  


## Input-output

write(): 将原子位置(atomic positions)保存到xyz格式的文件;

In [11]:
from ase.io import write
write('data/slab.xyz', slab)

> write()函数可支持的文件格式:  
`xyz`: Simple xyz-format  
`cube`:Gaussian cube file  
`pdb`: Protein data bank file  
`traj`: ASE’s own trajectory format  
`py`: Python script  

从文件中读取结构

In [12]:
from ase.io import read
slab_from_file = read('data/N2Cu.traj')

如果文件包含多个构型, write()函数只会返回最后一个构型

In [13]:
%%pass
read('slab.traj')      # last configuration
read('slab.traj', -1)  # same as above
read('slab.traj', 0)   # first configuration

## Visualization¶

可视化atoms对象的方法

In [14]:
%%pass
from ase.visualize import view
view(slab)

## Molecular dynamics¶

使用VelocityVerlet方法进行分子动力学模拟

In [15]:
from ase.md.verlet import VelocityVerlet
from ase import units

In [16]:
dyn = VelocityVerlet(molecule, dt=1.0 * units.fs)
for i in range(10):
    pot = molecule.get_potential_energy()
    kin = molecule.get_kinetic_energy()
    print('%2d: %.5f eV, %.5f eV, %.5f eV' % (i, pot + kin, pot, kin))
    dyn.run(steps=20)

 0: 0.44034 eV, 0.44034 eV, 0.00000 eV
 1: 0.43816 eV, 0.26289 eV, 0.17527 eV
 2: 0.44058 eV, 0.43142 eV, 0.00916 eV
 3: 0.43874 eV, 0.29292 eV, 0.14582 eV
 4: 0.44015 eV, 0.41839 eV, 0.02176 eV
 5: 0.43831 eV, 0.28902 eV, 0.14929 eV
 6: 0.43947 eV, 0.36902 eV, 0.07045 eV
 7: 0.43951 eV, 0.35507 eV, 0.08444 eV
 8: 0.43959 eV, 0.36221 eV, 0.07738 eV
 9: 0.43933 eV, 0.36044 eV, 0.07889 eV
